In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm
import h5py
import os

In [3]:
import h5py
import numpy as np

def inspect_h5(file_path):
    with h5py.File(file_path, 'r') as f:
        print("Keys in the file:")
        for key in f.keys():
            print(f"- {key}")
            data = f[key][()]
            print(f"  Shape: {data.shape}")
            print(f"  Data type: {data.dtype}")
            print(f"  Min value: {np.min(data)}")
            print(f"  Max value: {np.max(data)}")
            print(f"  Mean value: {np.mean(data)}")
            print(f"  Has NaNs: {np.isnan(data).any()}")
            print(f"  Has Infs: {np.isinf(data).any()}")
            print()

# Use the function
inspect_h5('raw_trajectories.h5')

Keys in the file:
- actions
  Shape: (2022, 1000, 2)
  Data type: float32
  Min value: -1.0
  Max value: 1.0
  Mean value: 0.41104790568351746
  Has NaNs: False
  Has Infs: False

- costs
  Shape: (2022, 1000)
  Data type: float32
  Min value: 0.0
  Max value: 1.0
  Mean value: 0.03068496473133564
  Has NaNs: False
  Has Infs: False

- observations
  Shape: (2022, 1000, 60)
  Data type: float32
  Min value: -19.96635627746582
  Max value: 19.938732147216797
  Mean value: 0.30550599098205566
  Has NaNs: False
  Has Infs: False

- rewards
  Shape: (2022, 1000)
  Data type: float32
  Min value: -0.028873039409518242
  Max value: 1.0294471979141235
  Mean value: 0.012595054693520069
  Has NaNs: False
  Has Infs: False

- terminals
  Shape: (2022, 1000)
  Data type: bool
  Min value: False
  Max value: False
  Mean value: 0.0
  Has NaNs: False
  Has Infs: False

- timeouts
  Shape: (2022, 1000)
  Data type: bool
  Min value: False
  Max value: True
  Mean value: 0.001
  Has NaNs: False
  Ha

In [48]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Print the name of the GPU
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

CUDA available: True
GPU Name: NVIDIA RTX A3000 Laptop GPU


In [49]:
def print_h5_structure(file_path):
    with h5py.File(file_path, 'r') as hf:
        def print_attrs(name, obj):
            print(name)
            for key, val in obj.attrs.items():
                print(f"    {key}: {val}")
        
        hf.visititems(print_attrs)

print_h5_structure('raw_trajectories.h5')

actions
costs
observations
rewards
terminals
timeouts


In [50]:
class SimpleUnet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.down1 = nn.Linear(in_channels, 256)
        self.down2 = nn.Linear(256, 512)
        self.down3 = nn.Linear(512, 1024)
        self.up1 = nn.Linear(1024, 512)
        self.up2 = nn.Linear(512, 256)
        self.up3 = nn.Linear(256, out_channels)
        
    def forward(self, x, t):
        b, seq_len, c = x.shape
        t = t.unsqueeze(1).expand(-1, seq_len)
        
        x1 = F.relu(self.down1(x))
        x2 = F.relu(self.down2(x1))
        x3 = F.relu(self.down3(x2))
        x = F.relu(self.up1(x3)) + x2
        x = F.relu(self.up2(x)) + x1
        return self.up3(x)

In [51]:
class DDPM(nn.Module):
    def __init__(self, eps_model, betas, n_T):
        super().__init__()
        self.eps_model = eps_model
        self.betas = betas
        self.n_T = n_T
        
        # Pre-compute different terms for closed form
        self.alphas = 1. - betas
        self.alphas_cumprod = torch.cumprod(self.alphas, axis=0)
        self.alphas_cumprod_prev = F.pad(self.alphas_cumprod[:-1], (1, 0), value=1.0)
        self.sqrt_recip_alphas = torch.sqrt(1.0 / self.alphas)
        self.sqrt_alphas_cumprod = torch.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - self.alphas_cumprod)
        self.posterior_variance = betas * (1. - self.alphas_cumprod_prev) / (1. - self.alphas_cumprod)
        
    def forward(self, x_0):
        b, seq_len, c = x_0.shape
        t = torch.randint(0, self.n_T, (b,)).to(x_0.device)
        noise = torch.randn_like(x_0)
        x_t = (
            self.sqrt_alphas_cumprod[t][:, None, None] * x_0 + 
            self.sqrt_one_minus_alphas_cumprod[t][:, None, None] * noise
        )
        return self.eps_model(x_t, t), noise

    @torch.no_grad()
    def p_sample(self, x, t, t_index):
        betas_t = self.betas[t][:, None, None]
        sqrt_one_minus_alphas_cumprod_t = self.sqrt_one_minus_alphas_cumprod[t][:, None, None]
        sqrt_recip_alphas_t = self.sqrt_recip_alphas[t][:, None, None]
        
        model_mean = sqrt_recip_alphas_t * (
            x - betas_t * self.eps_model(x, t) / sqrt_one_minus_alphas_cumprod_t
        )
        
        if t_index == 0:
            return model_mean
        else:
            posterior_variance_t = self.posterior_variance[t][:, None, None]
            noise = torch.randn_like(x)
            return model_mean + torch.sqrt(posterior_variance_t) * noise

    @torch.no_grad()
    def sample(self, seq_len, batch_size=16, channels=126):
        device = next(self.parameters()).device
        shape = (batch_size, seq_len, channels)
        img = torch.randn(shape, device=device)
        
        for i in tqdm(reversed(range(0, self.n_T)), desc='sampling loop time step', total=self.n_T):
            t = torch.full((batch_size,), i, device=device, dtype=torch.long)
            img = self.p_sample(img, t, i)
        return img

In [52]:
def load_data(file_path):
    with h5py.File(file_path, 'r') as hf:
        observations = hf['observations'][:]
        actions = hf['actions'][:]
        rewards = hf['rewards'][:]
        costs = hf['costs'][:]
        terminals = hf['terminals'][:]
        timeouts = hf['timeouts'][:]
    
    # Create next_observations by shifting observations
    next_observations = np.roll(observations, -1, axis=1)
    next_observations[:, -1] = observations[:, -1]  # Last step
    
    # Reshape 1D arrays to have a second dimension of 1
    rewards = rewards[..., np.newaxis]
    costs = costs[..., np.newaxis]
    terminals = terminals[..., np.newaxis]
    timeouts = timeouts[..., np.newaxis]
    
    # Combine all data into a single array
    trajectories = np.concatenate([
        observations,
        next_observations,
        actions,
        rewards,
        costs,
        terminals,
        timeouts
    ], axis=2)
    
    return torch.FloatTensor(trajectories)

In [53]:
@torch.no_grad()
def p_sample(model, x, t, t_index):
    betas_t = model.betas[t]
    sqrt_one_minus_alphas_cumprod_t = model.sqrt_one_minus_alphas_cumprod[t]
    sqrt_recip_alphas_t = model.sqrt_recip_alphas[t]
    
    model_mean = sqrt_recip_alphas_t * (x - betas_t * model.eps_model(x, t) / sqrt_one_minus_alphas_cumprod_t)
    
    if t_index == 0:
        return model_mean
    else:
        posterior_variance_t = model.posterior_variance[t]
        noise = torch.randn_like(x)
        return model_mean + torch.sqrt(posterior_variance_t) * noise

@torch.no_grad()
def p_sample_loop(model, shape):
    device = next(model.parameters()).device
    b = shape[0]
    img = torch.randn(shape, device=device)
    
    for i in tqdm(reversed(range(0, model.n_T)), desc='sampling loop time step', total=model.n_T):
        img = p_sample(model, img, torch.full((b,), i, device=device, dtype=torch.long), i)
    return img

@torch.no_grad()
def sample(model, image_size, batch_size=16, channels=3):
    return p_sample_loop(model, shape=(batch_size, channels, image_size))

In [54]:
class TrajectoryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            return self.data[idx]
        except Exception as e:
            print(f"Error loading item at index {idx}: {str(e)}")
            return None

In [58]:
class DDPMTrainer:
    def __init__(self, ddpm, dataset, device, learning_rate=1e-4, batch_size=32):
        self.ddpm = ddpm
        self.device = device
        self.dataset = dataset
        self.dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
        self.optimizer = torch.optim.Adam(ddpm.parameters(), lr=learning_rate)

    def save_model(self, epoch, loss, filename):
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.ddpm.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'loss': loss,
        }
        torch.save(checkpoint, filename)
        print(f"Model saved to {filename}")

    def load_model(self, filename):
        if os.path.isfile(filename):
            checkpoint = torch.load(filename)
            self.ddpm.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            epoch = checkpoint['epoch']
            loss = checkpoint['loss']
            print(f"Model loaded from {filename}")
            return epoch, loss
        else:
            print(f"No saved model found at {filename}")
            return 0, float('inf')

    def train(self, num_epochs, model_filename):
        start_epoch, best_loss = self.load_model(model_filename)
        
        for epoch in range(start_epoch, num_epochs):
            epoch_loss = 0.0
            for batch in tqdm(self.dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
                self.optimizer.zero_grad()
                batch = batch.to(self.device)
                predicted_noise, target_noise = self.ddpm(batch)
                loss = F.mse_loss(predicted_noise, target_noise)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            
            avg_loss = epoch_loss / len(self.dataloader)
            print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")
            
            if avg_loss < best_loss:
                best_loss = avg_loss
                self.save_model(epoch+1, best_loss, model_filename)
        
        print("Training completed.")

    def generate_trajectories(self, seq_len, batch_size, channels):
        print("Generating new trajectories...")
        generated_trajectories = self.ddpm.sample(seq_len=seq_len, batch_size=batch_size, channels=channels)
        print("Generated trajectories shape:", generated_trajectories.shape)
        return generated_trajectories

    def generate_and_save_trajectories(self, seq_len, total_trajectories, channels, save_path, batch_size=10):
        print(f"Generating and saving {total_trajectories} trajectories in batches of {batch_size}...")
        
        # Open a memory-mapped file for writing
        shape = (total_trajectories, seq_len, channels)
        memmap = np.memmap(save_path, dtype='float32', mode='w+', shape=shape)
        
        for start_idx in tqdm(range(0, total_trajectories, batch_size), desc="Generating batches"):
            end_idx = min(start_idx + batch_size, total_trajectories)
            current_batch_size = end_idx - start_idx
            
            # Generate a batch of trajectories
            with torch.no_grad():
                generated_batch = self.ddpm.sample(seq_len=seq_len, batch_size=current_batch_size, channels=channels)
            
            # Move to CPU and convert to numpy
            generated_batch_np = generated_batch.cpu().numpy()
            
            # Save to memmap
            memmap[start_idx:end_idx] = generated_batch_np
            
            # Explicitly delete to free up memory
            del generated_batch, generated_batch_np
            torch.cuda.empty_cache()
        
        # Flush the memmap to ensure all data is written
        memmap.flush()
        
        print(f"Generated trajectories saved to {save_path}")
        print(f"Shape of saved trajectories: {shape}")

In [ ]:
# Your main script would then look like this:
if name == "main":
    # Hyperparameters
    n_T = 1000
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    betas = torch.linspace(1e-4, 0.02, n_T).to(device)

    # Load data
    file_path = 'raw_trajectories.h5'
    trajectories = load_data(file_path)

    in_channels = trajectories.shape[2]

    print(f"Trajectory shape: {trajectories.shape}")
    print(f"Number of features per trajectory: {in_channels}")

    # Initialize models
    eps_model = SimpleUnet(in_channels, in_channels).to(device)
    ddpm = DDPM(eps_model, betas, n_T).to(device)

    # Create dataset
    dataset = TrajectoryDataset(trajectories)

    # Initialize trainer
    trainer = DDPMTrainer(ddpm, dataset, device)

    # Train DDPM
    model_filename = 'ddpm_model.pth'
    num_epochs = 100
    trainer.train(num_epochs, model_filename)

     # Generate and save new trajectories
    save_path = 'generated_trajectories.npy'
    trainer.generate_and_save_trajectories(
        seq_len=trajectories.shape[1], 
        total_trajectories=1000,  # This is the number of trajectories you want to generate
        channels=trajectories.shape[2],
        save_path=save_path,
        batch_size=10  # Adjust this based on your GPU memory
    )

    print("Process completed.")

In [2]:
import numpy as np

def inspect_generated_trajectories(file_path):
    memmap = np.memmap(file_path, dtype='float32', mode='r')
    
    print(f"Raw shape of generated trajectories: {memmap.shape}")
    print(f"Total number of elements: {memmap.size}")
    
    # Try to infer the correct shape
    total_elements = memmap.size
    num_trajectories = 1000  # As specified in your generation code
    seq_len = 1000  # From your original data
    
    if total_elements % (num_trajectories * seq_len) == 0:
        num_features = total_elements // (num_trajectories * seq_len)
        print(f"Inferred shape: ({num_trajectories}, {seq_len}, {num_features})")
        
        # Reshape the memmap
        trajectories = memmap.reshape((num_trajectories, seq_len, num_features))
        
        print(f"Data type: {trajectories.dtype}")
        print(f"Min value: {np.min(trajectories)}")
        print(f"Max value: {np.max(trajectories)}")
        print(f"Mean value: {np.mean(trajectories)}")
        print(f"Has NaNs: {np.isnan(trajectories).any()}")
        print(f"Has Infs: {np.isinf(trajectories).any()}")
        
        # Print statistics for each feature
        for i in range(num_features):
            print(f"\nFeature {i}:")
            print(f"  Min: {np.min(trajectories[:,:,i])}")
            print(f"  Max: {np.max(trajectories[:,:,i])}")
            print(f"  Mean: {np.mean(trajectories[:,:,i])}")
    else:
        print("Unable to infer the correct shape. Please check the generation process.")

# Use the function
inspect_generated_trajectories('generated_trajectories.npy')

Raw shape of generated trajectories: (126000000,)
Total number of elements: 126000000
Inferred shape: (1000, 1000, 126)
Data type: float32
Min value: -760.2193603515625
Max value: 938.1056518554688
Mean value: 1.6578973531723022
Has NaNs: False
Has Infs: False

Feature 0:
  Min: -461.65472412109375
  Max: 560.3864135742188
  Mean: 14.21492862701416

Feature 1:
  Min: -661.8180541992188
  Max: 938.1056518554688
  Mean: 12.59593391418457

Feature 2:
  Min: -742.8836059570312
  Max: 706.7652587890625
  Mean: 33.1786994934082

Feature 3:
  Min: -283.9253845214844
  Max: 299.37139892578125
  Mean: 7.248508453369141

Feature 4:
  Min: -300.78656005859375
  Max: 318.9814147949219
  Mean: 1.6598554849624634

Feature 5:
  Min: -268.4080810546875
  Max: 239.41165161132812
  Mean: -0.9450872540473938

Feature 6:
  Min: -266.3388366699219
  Max: 283.5146789550781
  Mean: 0.24941018223762512

Feature 7:
  Min: -254.8743438720703
  Max: 277.1191711425781
  Mean: -0.33862096071243286

Feature 8:
  Mi

In [3]:
import numpy as np
import h5py

def convert_and_save_generated_trajectories(generated_trajectories, output_file='generated_dataset.h5', seq_len=1000):
    """
    Convert generated trajectories to the format expected by the training algorithm,
    save as an HDF5 file, and return the dataset dictionary.
    
    :param generated_trajectories: numpy array of shape (num_trajectories, seq_len, num_features)
    :param output_file: name of the output HDF5 file
    :param seq_len: length of each trajectory
    :return: dictionary containing the dataset in the required format
    """
    num_trajectories, _, num_features = generated_trajectories.shape
    
    # Assuming the last 6 features are: rewards, costs, terminals, timeouts, actions (2D)
    observations = generated_trajectories[:, :, :60]
    next_observations = np.roll(observations, -1, axis=1)
    next_observations[:, -1] = observations[:, -1]  # Last step
    actions = generated_trajectories[:, :, -2:]
    rewards = generated_trajectories[:, :, -6]
    costs = generated_trajectories[:, :, -5]
    terminals = generated_trajectories[:, :, -4].astype(bool)
    timeouts = generated_trajectories[:, :, -3].astype(bool)
    
    # Create the dataset dictionary
    dataset = {
        'observations': observations.reshape(-1, 60),
        'next_observations': next_observations.reshape(-1, 60),
        'actions': actions.reshape(-1, 2),
        'rewards': rewards.flatten(),
        'costs': costs.flatten(),
        'terminals': terminals.flatten(),
        'timeouts': timeouts.flatten()
    }
    
    # Save the dataset in HDF5 format
    with h5py.File(output_file, 'w') as f:
        for key, value in dataset.items():
            f.create_dataset(key, data=value)
    
    print(f"Dataset converted and saved as '{output_file}'")
    
    return dataset

# Load the generated trajectories
generated_trajectories = np.memmap('generated_trajectories.npy', dtype='float32', mode='r').reshape(1000, 1000, 126)

# Convert to dataset format, save as HDF5, and get the dataset dictionary
dataset = convert_and_save_generated_trajectories(generated_trajectories)

print("Dataset ready for training")

Dataset converted and saved as 'generated_dataset.h5'
Dataset ready for training
